In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("tor.csv")

In [3]:
df.head()

,TransactionId,UserId,VisitYear,VisitMonth,VisitMode,AttractionId,Rating,ContinentId,RegionId,CountryId,CityId,Continent,Region,Country,CityName,Attraction,AttractionAddress,AttractionTypeId,AttractionType,VisitModeName
0,8,7567,2022,10,4,640,5,2,8,48,464,America,Northern America,Canada,Ontario,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,Friends
1,69,9804,2022,7,2,640,4,4,15,109,535,Australia & Oceania,Australia,Australia,Sydney,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,Couples
2,232,4371,2020,3,2,640,5,4,15,109,1434,Australia & Oceania,Australia,Australia,Melbourne,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,Couples
3,300,11099,2020,2,2,640,5,5,21,163,780,Europe,Western Europe,United Kingdom,Brighton,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,Couples
4,376,5021,2020,1,4,640,4,4,15,109,1434,Australia & Oceania,Australia,Australia,Melbourne,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,Friends


In [4]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [5]:
pip install imbalanced-learn



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder, LabelEncoder,StandardScaler
from category_encoders import TargetEncoder

In [6]:
selected_features = ["UserId", "VisitMode", "AttractionId", 
                     "ContinentId", "RegionId", "Attraction", "AttractionType", "AttractionTypeId"]

In [7]:
X = df[selected_features].copy()
y = df["VisitModeName"]

In [8]:
categorical_features = ["VisitMode", "AttractionType"]
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
encoded_features = ohe.fit_transform(X[categorical_features])
encoded_df = pd.DataFrame(encoded_features, columns=ohe.get_feature_names_out(categorical_features))

In [9]:
label_encoder = LabelEncoder()
y = pd.Series(label_encoder.fit_transform(y))

In [10]:
target_enc = TargetEncoder()
X["Attraction"] = target_enc.fit_transform(X["Attraction"], y)

In [11]:
bool_cols = X.select_dtypes(include=["bool"]).columns
X[bool_cols] = X[bool_cols].astype(int)

X = X.drop(columns=categorical_features)
X = pd.concat([X.reset_index(drop=True), encoded_df], axis=1)

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

y_smote = pd.DataFrame(y_smote, columns=['VisitModeName'])
balanced_tourism_data = pd.concat([X_smote, y_smote], axis=1)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost
from xgboost import XGBClassifier

In [13]:
from sklearn.metrics import accuracy_score,precision_score

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

In [15]:
rf_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
rf_model.fit(X_train, y_train)

c:\Users\Asus\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(max_depth=5, random_state=42)

In [16]:
rf_pred = rf_model.predict(X_test)
rf_pred
#EVALUATE THE RANDOM FOREST MODEL
accuracy_1 = accuracy_score(y_test, rf_pred)
precision_1 = precision_score(y_test, rf_pred, average='weighted')


In [17]:
print(accuracy_1,precision_1)

1.0 1.0


In [18]:
model2= XGBClassifier(n_estimators=50, max_depth=3, learning_rate=0.2, random_state=42)
model2.fit(X_train, y_train)
model2_pred = model2.predict(X_test)

In [19]:
accuracy = accuracy_score(y_test, model2_pred)
precision = precision_score(y_test, model2_pred, average='weighted')

In [20]:
print(accuracy,precision)

1.0 1.0


In [21]:
import joblib

In [22]:
joblib.dump(ohe,r"D:\Tourism\one_hot.pkl")

['D:\\Tourism\\one_hot.pkl']

In [23]:
joblib.dump(label_encoder,r"D:\Tourism\label_encoder.pkl")

['D:\\Tourism\\label_encoder.pkl']

In [24]:
joblib.dump(target_enc,r"D:\Tourism\target_encoder.pkl")

['D:\\Tourism\\target_encoder.pkl']

In [25]:
joblib.dump(model2,r"D:\Tourism\model2.pkl")

['D:\\Tourism\\model2.pkl']